# Dashboard

In [ ]:

# Imports 

import pandas as pd


# Get the possible data venues
venuekeysfile = "venue-keys.csv"
dfVenueKeys = pd.read_csv(venuekeysfile)

dfDash = pd.DataFrame(columns = ["Venue","Starts","Ends","Capture days","Last Voltage","Max awake (s)","Min awake (s)","Median awake(s)"])

for index, venueSensorDetails in dfVenueKeys.iterrows():
    try: 
        sensorMacOfSelection = venueSensorDetails['sensor_MAC']
        venueOfSelection = str(venueSensorDetails['venue_id'])
        dfTempDataSet = pd.read_csv('deviceData/'+ "venue_" + venueOfSelection + "_with_device_" + sensorMacOfSelection + '.csv' )
        dfTempDataSet['timestamp'] = pd.to_datetime(dfTempDataSet['timestamp'])
        dfTempDataSet['venue_id'] = venueSensorDetails['venue_id']
        dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature <= 100)]
        dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature > -10)] # eliminate rogue data
        #print("%s\t%s\t%s\t%d" %
        #    (venueOfSelection, dfTempDataSet.timestamp.min().strftime('%Y-%m-%d %H:%M'), dfTempDataSet.timestamp.max().strftime('%Y-%m-%d %H:%M'), len(dfTempDataSet.timestamp)/288))
        if (len(dfTempDataSet.timestamp)>0):
            first_time = dfTempDataSet.timestamp.min().strftime('%Y-%m-%d %H:%M')
            last_time = dfTempDataSet.timestamp.max().strftime('%Y-%m-%d %H:%M')
            days_captured = round(len(dfTempDataSet.timestamp.dropna())/288)
        else:
            first_time = "None"
            last_time = "None"
            days_captured = "None"
        try:
            volts = dfTempDataSet.voltage.dropna()
            last_voltage = volts.loc[volts != 1].iloc[-1]/10 
            # he's changed the voltage representation in later models.  Assume if < 1 it really is V, not V*10.
            if (last_voltage < 1):
                last_voltage = last_voltage * 10

        except: 
            last_voltage = "None" # there aren't any good voltage readings
        try:
            awake_time = dfTempDataSet.awake_time.dropna()
            # we want the intervals between the awake_time readings because those are cumulative,
            awake_interval = awake_time - awake_time.shift(1)
            # it starts from zero every time it reboots so correct negative intervals to just be whatever
            # the recorded time awake was.
            awake_interval[awake_interval<0] = awake_time
            print(type(awake_interval))
            max_awake = int(awake_interval.max())
            min_awake = int(awake_interval.min())
            median_awake = int(awake_interval.median())
        except: 
            max_awake = "None" # there aren't any good readings
            min_awake = "None"
            median_awake = "None"
        dfDash.loc[len(dfDash)] = ([venueOfSelection, first_time, last_time, days_captured, last_voltage, max_awake,min_awake,median_awake])
    except:
        print("Unknown dashboard error.")


In [ ]:

display(dfDash)
